<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Scripts%20de%20detecci%C3%B3n%20de%20Video/VideoDetectionDetectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conectar Colab a Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Install detectron2**

In [ ]:
%cd /content/
# install detectron2:
!git clone https://github.com/facebookresearch/detectron2
%cd /content/detectron2
!pip install -r requirements.txt
!python setup.py install
!pip install git+https://github.com/facebookresearch/fvcore.git
"""
%cd /content/
!git clone https://github.com/facebookresearch/iopath
%cd /content/iopath
!python setup.py install

%cd /content/
!git clone https://github.com/WoLpH/portalocker
%cd /content/portalocker
!python setup.py install
"""

**Import detectron2**

In [ ]:
#@markdown You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
from google.colab.patches import cv2_imshow
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

**Para descomprimir la carpeta de los modelos**

In [ ]:
# # Rutas de los archivos zip en Google Drive
# ruta_zip = '/content/drive/MyDrive/TFM/models/modelos.zip'

# # Rutas de las carpetas destino
# ruta_destino = '/content'

In [ ]:
# import os
# import zipfile

# # Crear la carpeta destino si no existen
# os.makedirs(ruta_destino, exist_ok=True)

# # Descomprimir los archivos zip
# with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
#     zip_ref.extractall(ruta_destino)

# print(f'Archivos descomprimidos en {ruta_destino}')

Archivos descomprimidos en /content


**Prueba de detección de obteos en video a tiempo real**

In [ ]:
import time
import os

# Ruta del modelo personalizado entrenado
custom_model_path = "/content/drive/MyDrive/TFM/models/Detectron2_n_fix.pth"

# Configuración para procesar video
initial_frames_to_test = 10  # Número de frames iniciales para medir el tiempo de procesamiento
video_path = "/content/drive/MyDrive/TFM/Cáncer Válvula Ileocecal.mp4"

# Obtener el nombre del modelo sin la extensión
model_name = os.path.splitext(os.path.basename(custom_model_path))[0]

# Definir la ruta de salida basada en el nombre del modelo
output_path = f"{model_name}_processed.mp4"

# Función para cargar el modelo Detectron2 personalizado
def load_custom_detectron_model(model_weights_path):
    cfg = get_cfg()
    # Configuración basada en tu tarea (por ejemplo, segmentación de instancias)
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Umbral de confianza
    cfg.MODEL.WEIGHTS = model_weights_path  # Cargar tus pesos entrenados
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Ajustar según el número de clases en tu modelo (ej. 1 si es binario)
    return DefaultPredictor(cfg)

# Cargar el modelo personalizado
predictor = load_custom_detectron_model(custom_model_path)

# Función para medir el tiempo de procesamiento y determinar la frecuencia óptima
def determine_processing_interval(video_path):
    cap = cv2.VideoCapture(video_path)

    # Obtener la información del video
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps

    # Medir el tiempo de procesamiento de los primeros `initial_frames_to_test` frames
    start_time = time.time()
    for _ in range(initial_frames_to_test):
        ret, frame = cap.read()
        if not ret:
            break
        # Procesar el frame con el modelo
        outputs = predictor(frame)
    processing_time = time.time() - start_time

    # Calcular el tiempo promedio por frame
    avg_processing_time_per_frame = processing_time / initial_frames_to_test

    # Calcular el intervalo de frames basado en el tiempo disponible
    max_processing_time_per_frame = 1 / fps  # Tiempo disponible por frame
    optimal_processing_interval = max(1, int(round(avg_processing_time_per_frame / max_processing_time_per_frame)))

    cap.release()

    return optimal_processing_interval

# Función para procesar video y guardar el resultado
def process_video(video_path, output_path, process_every_n_frames):
    cap = cv2.VideoCapture(video_path)

    # Obtener la información del video original
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Crear el objeto de escritura de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Procesar solo cada N frames
        if frame_count % process_every_n_frames == 0:
            outputs = predictor(frame)

            # Visualización y segmentación en el frame
            v = Visualizer(frame[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
            v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            frame = v.get_image()[:, :, ::-1]

            # Mostrar en Colab
            cv2_imshow(frame)

        # Escribir el frame procesado en el archivo de salida
        out.write(frame)

        frame_count += 1

    cap.release()
    out.release()

# Llamada a la función para determinar el intervalo de procesamiento óptimo
optimal_interval = determine_processing_interval(video_path)

print(f"Intervalo óptimo de procesamiento: cada {optimal_interval} frames")

# Llamada a la función para procesar el video con el intervalo óptimo
process_video(video_path, output_path, optimal_interval)

print(f"El video procesado se guardó en: {output_path}")

Streaming output truncated to the last 5000 lines.
Speed: 3.3ms preprocess, 10.0ms inference, 2.3ms postprocess per image at shape (1, 3, 544, 704)

0: 544x704 1 Polipo, 10.9ms
Speed: 3.3ms preprocess, 10.9ms inference, 2.2ms postprocess per image at shape (1, 3, 544, 704)

0: 544x704 1 Polipo, 9.9ms
Speed: 3.2ms preprocess, 9.9ms inference, 2.2ms postprocess per image at shape (1, 3, 544, 704)

0: 544x704 1 Polipo, 10.6ms
Speed: 3.4ms preprocess, 10.6ms inference, 2.4ms postprocess per image at shape (1, 3, 544, 704)

0: 544x704 1 Polipo, 15.5ms
Speed: 3.5ms preprocess, 15.5ms inference, 4.8ms postprocess per image at shape (1, 3, 544, 704)

0: 544x704 1 Polipo, 11.0ms
Speed: 3.2ms preprocess, 11.0ms inference, 2.5ms postprocess per image at shape (1, 3, 544, 704)

0: 544x704 (no detections), 15.9ms
Speed: 3.3ms preprocess, 15.9ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 704)

0: 544x704 1 Polipo, 14.3ms
Speed: 5.7ms preprocess, 14.3ms inference, 2.6ms postprocess p

In [ ]:
import shutil
import os

# Archivo de origen
src_file = output_path  # Asegúrate de que esto sea el archivo, no un directorio

# Directorio de destino
dest_dir = "/content/drive/MyDrive/TFM/Videos"

def copy_file(src, dest_dir):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    dest_file = os.path.join(dest_dir, os.path.basename(src))
    shutil.copy2(src, dest_file)

# Copiar el archivo de origen al directorio de destino
copy_file(src_file, dest_dir)

print(f"Archivo copiado de {src_file} a {dest_dir}")
